- Achmad Fariz Rizky Yanuar
- 03achmadfariz@gmail.com
- Universitas Sriwijaya

## Informasi Data

- Scrapping data review untuk aardy.com di trustpilot menggunakan library Selenium
- https://www.trustpilot.com/review/aardy.com
- Jumlah data: 18122

AARDY.com adalah platform untuk membandingkan dan membeli asuransi perjalanan dari berbagai penyedia. Mereka membantu menemukan polis dengan harga terbaik dan menyediakan ulasan serta peringkat untuk mempermudah perbandingan.

## Import Library

In [23]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf
from imblearn.over_sampling import RandomOverSampler
import joblib
import warnings
warnings.filterwarnings("ignore")

## Load Data

In [34]:
df = pd.read_csv('aardy_reviews_clean.csv')
df.dropna(subset=['content'], inplace=True)

## Exploratory Data Analysis (EDA)

In [35]:
print(f"Jumlah data: {len(df)}")

Jumlah data: 18122


Terdapat 18.122 samples yang digunakan.

In [33]:
df.head()

,title,content
0,Christianna Jeffries was great,Christianna Jeffries was great. So helpful and...
1,My daily experience is Hotmart website…,My daily experience is Hotmart website scam my...
3,Excellent Service,Melanie was an extremely helpful agent in assi...
4,Great representative,"Great representative, very personable and very..."
5,Melanie was very helpful.,Melanie was very helpful.. she got me all set ...


Featurenya terdiri dari `title` dan `content`. Feature `title` merujuk pada judul review dan feature `content` merujuk pada review yang diberikan oleh pengguna.

## Data Labelling

In [36]:
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
def get_sentiment(text):
    score = sia.polarity_scores(str(text))['compound']
    if score >= 0.05:
        return "positive"
    elif score <= -0.05:
        return "negative"
    else:
        return "neutral"
df['sentiment'] = df['content'].apply(get_sentiment)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Kita akan menggunakan VADER untuk melakukan pelabelan pada sentiment yang ada. Pelabelan dilakukan berdasarkan bobot yang telah ditentukan. Label berupa `positive`, `negative`, dan `neutral`.

In [37]:
df['sentiment'].value_counts()

,count
sentiment,
positive,16570
neutral,1240
negative,312


Terdapat 16570 sentiment `positive`, 1240 sentiment `neutral`, dan 312 sentiment `negative`.

In [47]:
df.isnull().sum()

,0
title,0
content,0
sentiment,0


Tidak ditemukan null values

## Preprocessing

In [13]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return ' '.join(words)

df['content_clean'] = df['content'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


- Mengubah semua huruf ke huruf kecil (lower()).

- Menghapus URL menggunakan regex.

- Menghapus tanda baca (seperti titik, koma, tanda seru, dll).

- Menghapus angka dari teks.

- Memecah teks jadi kata-kata (tokenisasi sederhana pakai split()).

- Menghapus stop words.

- Lemmatization (mengubah kata ke bentuk dasarnya).

## Split Data

In [14]:
X = df['content_clean']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

split datanya 80% training data dan 20% testing data

## Oversampling

In [15]:
ros = RandomOverSampler(random_state=42)
X_train_df = X_train.to_frame()
X_resampled, y_resampled = ros.fit_resample(X_train_df, y_train)
X_resampled = X_resampled['content_clean']

Karena data latih memiliki ketidakseimbangan antara jumlah sampel di tiap kelas, digunakan teknik oversampling dengan RandomOverSampler untuk menyeimbangkan distribusi kelas. Teknik ini bekerja dengan menggandakan data dari kelas minoritas sehingga jumlahnya seimbang dengan kelas mayoritas.

## Ekstraksi Fitur dan Pemodelan

Terdapat 3 skema:
1. TF-IDF + LR (80/20)
2. TF-IDF + Dense Layer (MLP) (80/20)
3. CountVectorizer + LR (70/30)

### Skema 1: TF-IDF + Logistic Regression

In [28]:
## Skema 1: TF-IDF + Logistic Regression
print("Skema 1: TF-IDF + Logistic Regression (Split 80/20)")
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_resampled)
X_test_tfidf = tfidf.transform(X_test)

model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train_tfidf, y_resampled)

print("Training - TF-IDF + Logistic Regression")
print(classification_report(y_resampled, model_lr.predict(X_train_tfidf)))

print("Testing - TF-IDF + Logistic Regression")
print(classification_report(y_test, model_lr.predict(X_test_tfidf)))

## Simpan model TF-IDF Logistic Regression sebagai Keras model
logistic_model_keras = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train_tfidf.shape[1],)),
    tf.keras.layers.Dense(3, activation='softmax', use_bias=True,
        kernel_initializer=tf.constant_initializer(model_lr.coef_.T),
        bias_initializer=tf.constant_initializer(model_lr.intercept_))
])
logistic_model_keras.save('model_logistic_regression.h5')

Skema 1: TF-IDF + Logistic Regression (Split 80/20)
Training - TF-IDF + Logistic Regression


              precision    recall  f1-score   support

    negative       0.99      1.00      0.99     13255
     neutral       0.98      0.99      0.99     13255
    positive       0.99      0.97      0.98     13255

    accuracy                           0.99     39765
   macro avg       0.99      0.99      0.99     39765
weighted avg       0.99      0.99      0.99     39765

Testing - TF-IDF + Logistic Regression
              precision    recall  f1-score   support

    negative       0.30      0.39      0.34        62
     neutral       0.61      0.93      0.74       248
    positive       0.99      0.95      0.97      3315

    accuracy                           0.94      3625
   macro avg       0.64      0.76      0.68      3625
weighted avg       0.95      0.94      0.94      3625



Pada proses pelatihan dengan skema TF-IDF + Logistic Regression (split 80/20), model menunjukkan performa yang sangat baik dengan akurasi sebesar 99%. Precision, recall, dan f1-score di ketiga kelas (negative, neutral, positive) juga sangat tinggi dan seimbang, yang menunjukkan bahwa model mampu mengenali pola data latih dengan sangat baik. Hal ini menandakan model berhasil belajar dari data training tanpa kesulitan berarti.

Namun, saat diuji pada data testing, performa model menurun secara signifikan untuk kelas tertentu. Meskipun akurasi keseluruhan masih tinggi (94%), model tampak sangat dominan dalam mengenali kelas positif, tetapi kesulitan dalam mengenali kelas negatif, dengan precision hanya 0.30 dan f1-score sebesar 0.34. Hal ini menunjukkan bahwa model belum mampu melakukan generalisasi dengan baik pada kelas minoritas, kemungkinan besar akibat ketidakseimbangan data.

### Skema 2: TF-IDF + Dense Layer (MLP)

In [27]:
## Skema 2: TF-IDF + Dense Layer (MLP)
print("Skema 2: TF-IDF + Dense Layer (MLP) (Split 80/20)")
le = LabelEncoder()
y_train_enc = le.fit_transform(y_resampled)
y_test_enc = le.transform(y_test)

X_train_arr = X_train_tfidf.toarray()
X_test_arr = X_test_tfidf.toarray()
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

model_dense = Sequential()
model_dense.add(Dense(128, activation='relu', input_shape=(X_train_arr.shape[1],)))
model_dense.add(Dense(64, activation='relu'))
model_dense.add(Dense(3, activation='softmax'))
model_dense.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_dense.fit(X_train_arr, y_train_cat, epochs=5, batch_size=32, validation_split=0.1)

print("Training - TF-IDF + Dense Layer")
y_pred_train = np.argmax(model_dense.predict(X_train_arr), axis=1)
print(classification_report(y_train_enc, y_pred_train, target_names=le.classes_))

print("Testing - TF-IDF + Dense Layer")
y_pred_test = np.argmax(model_dense.predict(X_test_arr), axis=1)
print(classification_report(y_test_enc, y_pred_test, target_names=le.classes_))

Skema 2: TF-IDF + Dense Layer (MLP) (Split 80/20)
Epoch 1/5
1119/1119 ━━━━━━━━━━━━━━━━━━━━ 35s 29ms/step - accuracy: 0.8768 - loss: 0.3243 - val_accuracy: 0.9872 - val_loss: 0.0518
Epoch 2/5
1119/1119 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - accuracy: 0.9976 - loss: 0.0090 - val_accuracy: 0.9990 - val_loss: 0.0070
Epoch 3/5
1119/1119 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.9995 - loss: 0.0022 - val_accuracy: 0.9990 - val_loss: 0.0021
Epoch 4/5
1119/1119 ━━━━━━━━━━━━━━━━━━━━ 22s 16ms/step - accuracy: 0.9995 - loss: 0.0016 - val_accuracy: 1.0000 - val_loss: 8.4940e-05
Epoch 5/5
1119/1119 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - accuracy: 0.9998 - loss: 7.4270e-04 - val_accuracy: 1.0000 - val_loss: 4.2672e-04
Training - TF-IDF + Dense Layer
1243/1243 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     13255
     neutral       1.00      1.00      1.00     13255
    positive       1.00      1.00      1.00  

Model MLP yang dilatih menggunakan representasi TF-IDF menunjukkan performa yang baik selama pelatihan. Akurasi training mencapai hampir 100% hanya dalam 5 epoch, dan performa pada data validasi juga sangat tinggi dengan akurasi mencapai 100% di epoch terakhir. Hasil evaluasi pada data training pun mencerminkan hal ini, dengan precision, recall, dan f1-score sempurna (1.00) di semua kelas. Ini menunjukkan bahwa model sangat kuat dalam mengenali pola pada data latih.

Namun, saat diuji pada data testing, performa model menunjukkan penurunan terutama pada kelas negatif. Meskipun akurasi keseluruhan masih sangat tinggi (96%) dan prediksi pada kelas positif tetap sangat baik (f1-score 0.98), f1-score untuk kelas negatif hanya mencapai 0.33. Hal ini mengindikasikan bahwa model overfitting terhadap data training dan kesulitan dalam melakukan generalisasi, khususnya pada kelas dengan jumlah data lebih sedikit atau yang kurang representatif di data pelatihan.

### Skema 3: CountVectorizer + Logistic Regression (new data split 70/30)

In [24]:
# Split data
X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42
)

# Oversampling
ros = RandomOverSampler(random_state=42)
X_train_70_df = X_train_70.to_frame()
X_resampled_70, y_resampled_70 = ros.fit_resample(X_train_70_df, y_train_70)
X_resampled_70 = X_resampled_70['content_clean']

# Vectorization
count_vec_70 = CountVectorizer()
X_train_count_70 = count_vec_70.fit_transform(X_resampled_70)
X_test_count_30 = count_vec_70.transform(X_test_30)

# Model training
model_lr_count_70 = LogisticRegression(max_iter=1000)
model_lr_count_70.fit(X_train_count_70, y_resampled_70)

# Evaluation
print("\n[Training] CountVectorizer + Logistic Regression")
print(classification_report(y_resampled_70, model_lr_count_70.predict(X_train_count_70)))

print("\n[Testing] CountVectorizer + Logistic Regression")
print(classification_report(y_test_30, model_lr_count_70.predict(X_test_count_30)))

# Simpan model dan vectorizer

joblib.dump(model_lr_count_70, 'model_lr_count_70.pkl')
joblib.dump(count_vec_70, 'count_vectorizer_70.pkl')


[Training] CountVectorizer + Logistic Regression
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     11599
     neutral       0.99      1.00      1.00     11599
    positive       1.00      0.99      0.99     11599

    accuracy                           1.00     34797
   macro avg       1.00      1.00      1.00     34797
weighted avg       1.00      1.00      1.00     34797


[Testing] CountVectorizer + Logistic Regression
              precision    recall  f1-score   support

    negative       0.40      0.38      0.39        94
     neutral       0.69      0.93      0.79       372
    positive       0.99      0.97      0.98      4971

    accuracy                           0.95      5437
   macro avg       0.69      0.76      0.72      5437
weighted avg       0.96      0.95      0.95      5437



['count_vectorizer_70.pkl']

Model yang dilatih dengan menggunakan CountVectorizer dan Logistic Regression menunjukkan performa yang sangat tinggi pada data pelatihan. Dengan akurasi 100% dan nilai precision, recall, serta f1-score yang hampir sempurna di ketiga kelas, model tampak sangat kuat dalam mempelajari pola dari data latih. Ini menandakan bahwa model berhasil mengoptimalkan representasi kata dari CountVectorizer dan mampu memetakan dengan baik ke masing-masing label.

Namun, saat diuji pada data testing, performa model menurun secara signifikan pada kelas negatif. Akurasi total masih tinggi di angka 95%, tetapi f1-score untuk kelas negatif hanya 0.39. Sebaliknya, model tetap sangat baik dalam mengenali kelas positif, dengan f1-score mencapai 0.98. Hal ini mengindikasikan bahwa model mengalami overfitting dan kesulitan dalam melakukan generalisasi, terutama terhadap kelas minoritas seperti `negative`.

## Inference

In [38]:
new_text = ["i had a great experience, the customer service was amazing"]

In [43]:
## Inference Contoh (Skema 2: MLP)
new_text_vec = tfidf.transform(new_text).toarray()
mlp_model_loaded = tf.keras.models.load_model('model_dense_layer.h5')
pred_class = mlp_model_loaded.predict(new_text_vec)
pred_label = le.classes_[np.argmax(pred_class)]
print(f"Prediksi (MLP): {pred_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
Prediksi (MLP): positive


In [44]:
## Inference Contoh (Skema 1: TF-IDF + Logistic Regression)
logistic_loaded_model = tf.keras.models.load_model('model_logistic_regression.h5')
log_pred = logistic_loaded_model.predict(new_text_vec)
pred_label_log = le.classes_[np.argmax(log_pred)]
print(f"Prediksi (LogReg): {pred_label_log}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Prediksi (LogReg): positive


In [46]:
# Skema 3 (CountVectorizer + Logistic Regression)
# Load model dan vectorizer
loaded_model = joblib.load('model_lr_count_70.pkl')
loaded_vectorizer = joblib.load('count_vectorizer_70.pkl')

# Transform dan prediksi
new_text_transformed = loaded_vectorizer.transform(new_text)
pred_count = loaded_model.predict(new_text_transformed)

print(f"\nPrediksi (CountVec + LogReg): {pred_count[0]}")


Prediksi (CountVec + LogReg): positive


Pada tahap inference, ketiga model—baik MLP (Skema 2), Logistic Regression dengan TF-IDF (Skema 1), maupun Logistic Regression dengan CountVectorizer (Skema 3)—mampu memprediksi label dari input teks baru "**i had a great experience, the customer service was amazing**" dengan hasil yang konsisten, yaitu "positive". Teks terlebih dahulu diubah ke bentuk numerik menggunakan TF-IDF atau CountVectorizer, kemudian dimasukkan ke model yang telah dilatih dan disimpan sebelumnya. Proses ini menunjukkan bahwa ketiga model dapat digunakan untuk klasifikasi teks secara langsung setelah pelatihan, meskipun muncul beberapa peringatan teknis terkait format penyimpanan model, yang tidak mempengaruhi hasil prediksi.